## 1) Шаги 1-5: Подготовка среды и Android SDK


In [ ]:
%%bash
set -euxo pipefail
export DEBIAN_FRONTEND=noninteractive
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 1/6] base packages $(date -u)"
apt-get update
apt-get install -y --no-install-recommends   ca-certificates git curl unzip zip openjdk-17-jdk-headless

echo "[STEP 2/6] android cmdline-tools $(date -u)"
mkdir -p "$ANDROID_SDK_ROOT/cmdline-tools"
if [ ! -x "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" ]; then
  curl -fL --retry 5 --retry-delay 5     https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip     -o /tmp/android-cmdline-tools.zip
  unzip -q /tmp/android-cmdline-tools.zip -d "$ANDROID_SDK_ROOT/cmdline-tools"
  rm -rf "$ANDROID_SDK_ROOT/cmdline-tools/latest"
  mv "$ANDROID_SDK_ROOT/cmdline-tools/cmdline-tools" "$ANDROID_SDK_ROOT/cmdline-tools/latest"
fi

echo "[STEP 3/6] accept sdk licenses $(date -u)"
set +o pipefail
timeout 900 bash -lc 'yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/android-sdk --licenses --verbose'   > /tmp/sdk-licenses.log 2>&1
licenses_rc=$?
set -o pipefail
tail -n 120 /tmp/sdk-licenses.log || true
if [ "$licenses_rc" -eq 124 ]; then
  echo "sdkmanager --licenses timeout (15 min)"
  exit 124
fi
if [ "$licenses_rc" -ne 0 ]; then
  echo "sdkmanager --licenses failed: $licenses_rc"
  exit "$licenses_rc"
fi

echo "[STEP 4/6] install sdk packages $(date -u)"
set +o pipefail
yes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT"   "platform-tools" "platforms;android-34" "build-tools;34.0.0"
sdk_rc=${PIPESTATUS[1]}
set -o pipefail
if [ "$sdk_rc" -ne 0 ]; then
  echo "sdkmanager packages failed: $sdk_rc"
  exit "$sdk_rc"
fi

echo "[STEP 5/6] write env $(date -u)"
cat > /content/tgwebmobile_env.sh <<'ENVEOF'
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export ANDROID_SDK_ROOT=/content/android-sdk
export ANDROID_HOME=$ANDROID_SDK_ROOT
export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
ENVEOF

echo "[DONE] steps 1-5 $(date -u)"


## 2) Шаги 6-6.5: Синхронизация репозитория и Firebase-файлы


In [ ]:
import json
import os
import shutil
import subprocess
import urllib.error
import urllib.request

REPO = '/content/tgwebmobile'

# Авто-получение Firebase файлов с push-сервера (без ручного upload)
PUSH_SHARED_SECRET = 'flygram_push_2026'
SERVER_BASES = [
    'https://sosiskibot.ru/flygram/push',
    'http://91.233.168.135:8081/flygram/push',
    'http://192.168.1.109:8081/flygram/push',
]

sync_script = """
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[STEP 6/6] clone/sync project + local.properties $(date -u)"
cd /content
if [ ! -d tgwebmobile ]; then
  git clone https://github.com/Perdonus/tgwebmobile.git
fi

cd /content/tgwebmobile
git fetch origin main
git reset --hard origin/main

cat > local.properties <<EOF2
sdk.dir=/content/android-sdk
EOF2

chmod +x gradlew
git --no-pager log --oneline -n 5
"""
subprocess.run(['bash', '-lc', sync_script], check=True)

os.makedirs(f'{REPO}/app', exist_ok=True)
os.makedirs(f'{REPO}/firebase', exist_ok=True)

google_dst = f'{REPO}/app/google-services.json'
admin_dst = f'{REPO}/firebase/firebase-adminsdk.json'

def is_google_services_json(path: str) -> bool:
    try:
        with open(path, 'r', encoding='utf-8') as f:
            root = json.load(f)
        return isinstance(root, dict) and 'project_info' in root and 'client' in root
    except Exception:
        return False

def is_admin_sdk_json(path: str) -> bool:
    try:
        with open(path, 'r', encoding='utf-8') as f:
            root = json.load(f)
        return isinstance(root, dict) and root.get('type') == 'service_account' and 'private_key' in root
    except Exception:
        return False

def download_from_server(urls, dst, validator, label):
    errors = []
    headers = {
        'X-FlyGram-Key': PUSH_SHARED_SECRET,
        'Accept': 'application/json',
        'User-Agent': 'flygram-colab-build/1.0',
    }

    for u in urls:
        try:
            print(f'Trying {label}: {u}')
            req = urllib.request.Request(u, headers=headers)
            with urllib.request.urlopen(req, timeout=25) as resp:
                status = getattr(resp, 'status', 200)
                payload = resp.read()
            if status != 200:
                raise RuntimeError(f'HTTP {status}')
            if not payload:
                raise RuntimeError('empty response')

            tmp = '/tmp/flygram_firebase_tmp.json'
            with open(tmp, 'wb') as f:
                f.write(payload)

            if not validator(tmp):
                raise RuntimeError('invalid JSON shape')

            shutil.copy2(tmp, dst)
            print(f'OK {label}: {u} -> {dst}')
            return True
        except urllib.error.HTTPError as e:
            errors.append(f'{u} -> HTTP {e.code}')
            print(f'HTTP {e.code} {u}')
        except Exception as e:
            errors.append(f'{u} -> {type(e).__name__}: {e}')
            print(f'Fail {u}: {e}')

    print(f'\nFailed to fetch {label} from server URLs:')
    for err in errors:
        print(' -', err)
    return False

# Основной новый маршрут /v1/files/* + старый /files/* как fallback
google_urls = []
admin_urls = []
for base in SERVER_BASES:
    google_urls.append(f'{base}/v1/files/google-services.json')
    admin_urls.append(f'{base}/v1/files/firebase-adminsdk.json')
for base in SERVER_BASES:
    google_urls.append(f'{base}/files/google-services.json')
    admin_urls.append(f'{base}/files/firebase-adminsdk.json')

ok_google = download_from_server(google_urls, google_dst, is_google_services_json, 'google-services.json')
ok_admin = download_from_server(admin_urls, admin_dst, is_admin_sdk_json, 'firebase-adminsdk.json')

if not (ok_google and ok_admin):
    raise RuntimeError(
        'Server Firebase files are unavailable. '        'Check flygram-push endpoint /v1/files/* and X-FlyGram-Key.'
    )

with open(google_dst, 'r', encoding='utf-8') as f:
    g = json.load(f)
with open(admin_dst, 'r', encoding='utf-8') as f:
    admin = json.load(f)

project_id = admin.get('project_id', '')
package_name = ''
try:
    client = (g.get('client') or [])[0]
    package_name = client.get('client_info', {}).get('android_client_info', {}).get('package_name', '')
except Exception:
    pass

env_path = '/content/tgwebmobile_push_env.sh'
with open(env_path, 'w', encoding='utf-8') as f:
    f.write(f'export FCM_PROJECT_ID="{project_id}"\n')
    f.write(f'export FCM_SERVICE_ACCOUNT_JSON="{admin_dst}"\n')
    f.write('export PUSH_BIND_HOST="192.168.1.109"\n')
    f.write('export PUSH_PUBLIC_HOST="91.233.168.135"\n')
    f.write('export PUSH_PORT="8081"\n')
    f.write('export PUSH_BASE_PATH="/flygram/push"\n')
    f.write(f'export PUSH_SHARED_SECRET="{PUSH_SHARED_SECRET}"\n')

print('\n[DONE] steps 6-6.5')
print(' -', google_dst)
print(' -', admin_dst)
print('Project ID:', project_id or '(not found)')
print('Android package:', package_name or '(not found)')
print('Env:', env_path)


## 3) Шаги 7-9: Сборка, проверка и скачивание APK


In [ ]:
import hashlib
import os
import subprocess

build_script = """
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[BUILD] start $(date -u)"
cd /content/tgwebmobile
./gradlew --no-daemon :app:assembleDebug

echo "[VERIFY] apk checks"
APK=/content/tgwebmobile/app/build/outputs/apk/debug/app-debug.apk
test -f "$APK"
ls -lh "$APK"
sha256sum "$APK"

echo "[DONE] steps 7-9 $(date -u)"
"""
subprocess.run(['bash', '-lc', build_script], check=True)

apk = '/content/tgwebmobile/app/build/outputs/apk/debug/app-debug.apk'
if not os.path.isfile(apk):
    raise RuntimeError(f'APK not found: {apk}')

h = hashlib.sha256()
with open(apk, 'rb') as f:
    for chunk in iter(lambda: f.read(1024 * 1024), b''):
        h.update(chunk)

print('APK:', apk)
print('SHA256:', h.hexdigest())

try:
    from google.colab import files
    files.download(apk)
    print('Download started in browser.')
except Exception as e:
    print('Auto-download unavailable. Manual path:', apk)
    print('Error:', e)
